

## Jubaclassifier Hands-on
- 主に"機械学習入門 実践Jubatusマスター"の内容について、コードを実行しながら説明
- データは書籍とは違うもの(default of credit card clients Data Set)を使います

## 手順
1. Jubatusの起動
1. データの読み込み、Datumに変換
1. 作成したDatumをJubatusサーバに投入
1. 学習モデルを用いて分類を行う
1. 結果の分析をしてみる
1. 前処理をしてみる
1. 指標のトレードオフ
1. まとめ


In [ ]:
## データの読み込み、確認
import pandas as pd
df = pd.read_csv("data/default_train.csv") #データの読み込み
print(df.head())

## 1. Jubatusを起動

- ターミナルに戻り、  `$ jubaclassifier -f config/linear.json& -t 1000`と入力します
- 今回は線形分類器(AROW)を使ってみます

## 2. データを読み込み、Datumに変換

In [ ]:
# データを読み込む関数
def read_dataset(path):
    df = pd.read_csv(path)
    labels = df['Y'].tolist()
    df = df.drop('Y', axis=1)
    features = df.as_matrix().astype(float)
    columns = df.columns.tolist()
    return features, labels, columns

features_train, labels_train, columns = read_dataset("data/default_train.csv")
print("columns : {}".format(columns))
print("features : {}".format(features_train))

In [ ]:
## 学習用データをDatum形式に変換
from jubatus.common import Datum
features_train, labels_train, columns = read_dataset('data/default_train.csv')
train_data = []
for x, y in zip(features_train, labels_train):
    d = Datum({key: float(value) for key, value in zip(columns, x)})
    train_data.append([str(y), d])

## 3. 作成したデータをJubatusサーバに投入

In [ ]:
from jubatus.classifier.client import Classifier
client = Classifier('127.0.0.1', 9199, '') # Jubatusサーバのホストとポートを指定する
client.clear() #過去の学習結果を一度初期化する(任意)
client.train(train_data) # 学習を実行

## 4. 学習したモデルを用いて分類を行う

- 作った学習モデルを使って、実際に分類をしてみる
- テストには`data/default_test.csv`を用いる



In [ ]:
# テスト用Datumリストを作る
features_test, labels_test, columns = read_dataset('data/default_test.csv')
test_data = []
for x, y in zip(features_test, labels_test):
    d = Datum({key: float(value) for key, value in zip(columns, x)})
    test_data.append(d)

In [ ]:
# テストをする
results = client.classify(test_data)

In [ ]:
# 結果の確認
print(results[0])

## 5. 結果の分析を行う
- 先ほどのスコアの大きい方を分類結果として返す`get_most_likely`関数を作る
- 結果の混合行列, accuracy, precision, recall, F-valueを算出する

In [ ]:
# 結果を分析する(スコアの大きい方のラベルを選ぶだけ)
def get_most_likely(result):
    return max(result, key = lambda x: x.score).label

In [ ]:
# 結果を分析する関数
def analyze_results(labels, results, pos_label="1", neg_label="0"):
    tp, fp, tn, fn = 0, 0, 0, 0
    for label, result in zip(labels, results):
        estimated = get_most_likely(result)
        label = str(label)
        estimated = str(estimated)
        if label == pos_label and label == estimated:
            tp += 1 #正例と判断し正しい : True Positive
        elif label == pos_label and label != estimated:
            fn += 1 #負例と判断し間違った : False Negative
        elif labels != pos_label and label == estimated:
            tn += 1 #負例と判断し正しい : True Negative
        else:
            fp += 1 #正例と判断し間違った : False Positive
    accuracy = float(tp + tn) / float(tp + tn + fp + fn) 
    precision = float(tp) / float(tp + fp) 
    recall = float(tp) / float(tp + fn)
    f_value = 2.0 * recall * precision / (recall + precision)
    # confusion matrix
    confusion = pd.DataFrame([[tp, fp], [fn, tn]], 
                             index=[pos_label, neg_label], columns=[pos_label, neg_label])
    return confusion, accuracy, precision, recall, f_value

In [ ]:
# 結果の確認
confusion, accuracy, precision, recall, f_value = analyze_results(labels_test, results)
print('confusion matrix\n{0}\n'.format(confusion))
print('metric    : score')
print('accuracy  : {0:.3f}'.format(accuracy))
print('precision : {0:.3f}'.format(precision))
print('recall    : {0:.3f}'.format(recall))
print('f_value   : {0:.3f}'.format(f_value))

## 6. 前処理をしてみる
Recallを上げるために => アンダーサンプリング

In [ ]:
# 各ラベルの学習用データの数
print(df[df["Y"]==0].shape)
print(df[df["Y"]==1].shape)

In [ ]:
import random
random.seed(42) # シードで乱数を固定(再現性を得たい場合に実行)
def under_sampling(features, labels, reduce_label, reduce_rate=0.2):
    # reduce_rateの割合で、reduce_labelを残す(残りは捨てる)
    sampled_features, sampled_labels = [], []
    for feature, label in zip(features, labels):
        label = str(label)
        if label != reduce_label or random.random() < reduce_rate:
            sampled_features.append(feature)
            sampled_labels.append(label)
    return sampled_features, sampled_labels

In [ ]:
# アンダーサンプリング
reduce_rate = 0.2
sampled_features_train, sampled_labels_train = under_sampling(features_train, labels_train, 
                                                              reduce_label="0",reduce_rate=reduce_rate)
# 学習用Datumリストを作る
sampled_train_data = []
for x, y in zip(sampled_features_train, sampled_labels_train):
    d = Datum({key: float(value) for key, value in zip(columns, x)})
    sampled_train_data.append([str(y), d])
# 学習をする
client = Classifier('127.0.0.1', 9199, '')
client.clear()
client.train(sampled_train_data)
# テストをする
results = client.classify(test_data)


In [ ]:
# 結果を分析する
confusion, accuracy, precision, recall, f_value = analyze_results(labels_test, results)
print('confusion matrix\n{0}\n'.format(confusion))
print('metric    : score')
print('accuracy  : {0:.3f}'.format(accuracy))
print('precision : {0:.3f}'.format(precision))
print('recall    : {0:.3f}'.format(recall))
print('f_value   : {0:.3f}'.format(f_value))

## 7. 指標のトレードオフ
PrecisionとRecallは、トレードオフの関係にある

|metric|score<br>normal|score<br>under sampling|
|---|---|---|
|Accuracy|0.803|0.542|
|Precision|**0.568**|**0.276**|
|Recall|**0.291**|**0.716**|
|f_value|0.385|0.398|



In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# アンダーサンプリングの割合を変えていく
rs = np.linspace(0,1.0,11)
precisions = []
recalls = []


In [ ]:
for r in rs:
    sampled_features_train, sampled_labels_train = under_sampling(features_train, labels_train, reduce_label="0",reduce_rate=r)
    # 学習用Datumリストを作る
    sampled_train_data = []
    for x, y in zip(sampled_features_train, sampled_labels_train):
        d = Datum({key: float(value) for key, value in zip(columns, x)})
        sampled_train_data.append([str(y), d])
    # 学習をする
    client = Classifier('127.0.0.1', 9199, '')
    client.clear()
    client.train(sampled_train_data)
    # テストをする
    results = client.classify(test_data)
    confusion, accuracy, precision, recall, f_value = analyze_results(labels_test, results)
    precisions.append(precision)
    recalls.append(recall)
#     print(confusion)
    print("rate:{:.1f} precision:{:.2f} recall:{:.2f} accuracy:{:.2f} f-value:{:.2f}".format(r, precision, recall, accuracy, f_value))

In [ ]:
# plotを行う
plt.plot(rs, precisions,"o-",alpha=0.5,label="precision")
plt.plot(rs,recalls,"o-",alpha=0.5,label="recall")
plt.xlim(0,1)
plt.ylim(0,1)
plt.legend()
plt.show()

## まとめ

- Jubatusを使って分類をやってみた
- 分類の評価指標はたくさんあるので、**目的に合わせて選ぶ**
- データをうまく処理することで、線形分類器でも精度を上げることが可能
 